In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ToxicityPrompts/mWildGuardMix-train-tower-nllb-v2.1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/993 [00:00<?, ?B/s]

data/train-00000-of-00006.parquet:   0%|          | 0.00/71.1M [00:00<?, ?B/s]

data/train-00001-of-00006.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00002-of-00006.parquet:   0%|          | 0.00/243M [00:00<?, ?B/s]

data/train-00003-of-00006.parquet:   0%|          | 0.00/293M [00:00<?, ?B/s]

data/train-00004-of-00006.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

data/train-00005-of-00006.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1474903 [00:00<?, ? examples/s]

In [ ]:
df = ds['train'].to_pandas()

In [13]:
df = df[df['language'] == 'English']

In [14]:
import pandas as pd

category_column = "prompt_harm_label"
sample_size = 500

unique_categories = df[category_column].unique()
num_categories = len(unique_categories)
samples_per_category = sample_size // num_categories
remaining_samples = sample_size % num_categories

# Create an empty DataFrame to store the balanced subset
sampled_df = pd.DataFrame()

# Sample from each category
for i, category in enumerate(unique_categories):
    category_df = df[df[category_column] == category]
    # Distribute remaining samples to the first few categories
    current_sample_size = samples_per_category + (1 if i < remaining_samples else 0)

    # Ensure we don't try to sample more rows than are available
    current_sample_size = min(current_sample_size, len(category_df))

    # Sample and append to the result DataFrame
    sampled_rows = category_df.sample(n=current_sample_size, random_state=711) # Use a fixed random_state for reproducibility
    sampled_df = pd.concat([sampled_df, sampled_rows])

print(f"Successfully created a subset of size {len(sampled_df)}.")

Successfully created a subset of size 500.


In [20]:
sampled_df.reset_index(drop=True, inplace=True)
sampled_df

sampled_df.to_json('prompting_subset.json', index=False, lines=True, orient='records')